In [2]:
import pandas as pd
import sqlite3

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_14384\2059591896.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
connection = sqlite3.connect("olist.db")

df = pd.read_sql_query("SELECT * FROM TrainingDataset",connection)

connection.close()

In [4]:
from sklearn.model_selection import train_test_split

y = df['score']
X = df[['produit_recu']]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [5]:
print("X_train shape:", X_train.shape)

X_train shape: (78675, 1)


In [6]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(X_train, y_train)

LogisticRegression()

In [7]:
from sklearn.metrics import recall_score, accuracy_score, f1_score

recall = recall_score(y_train, model.predict(X_train))
accuracy = accuracy_score(y_train, model.predict(X_train))
f1 = f1_score(y_train, model.predict(X_train))

print(f'le recall est de {recall},accuracy de {accuracy} et le f1 de {f1}')

le recall est de 0.9852335996126845,accuracy de 0.6394661582459485 et le f1 de 0.7594249607735041


In [8]:
from sklearn.metrics import recall_score, accuracy_score, f1_score

recall = recall_score(y_test, model.predict(X_test))
accuracy = accuracy_score(y_test, model.predict(X_test))
f1 = f1_score(y_test, model.predict(X_test))

print(f'le recall est de {recall},accuracy de {accuracy} et le f1 de {f1}')

le recall est de 0.9854704595185996,accuracy de 0.6439574965682038 et le f1 de 0.7627790386504523


### Modele avec délais

In [9]:
from sklearn.model_selection import train_test_split

y = df['score']
X = df[['produit_recu', 'temps_livraison', 'retard_livraison']]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [10]:
from sklearn.impute import SimpleImputer

# Je choisis la stratgéie de remplacer par le maximu
imputer_temps_livraison = SimpleImputer(strategy="constant",fill_value=X_train.temps_livraison.max())

# j'ajuste mon transformeur pour qu'il prenne la valeur du max sur le jeu de train
imputer_temps_livraison.fit(X_train[["temps_livraison"]])

# je remplace les valeurs manquantes du jeu de train par le max calculé (sur le jeu de train)
X_train[["temps_livraison"]] = imputer_temps_livraison.transform(X_train[["temps_livraison"]])

# je remplace les valeurs manquantes du jeu de test par le max calculé (sur le jeu de train)
X_test[["temps_livraison"]] = imputer_temps_livraison.transform(X_test[["temps_livraison"]])

In [11]:
# Je choisis la stratgéie de remplacer par le maximum
imputer_retard_livraison = SimpleImputer(strategy="constant",fill_value=X_train.retard_livraison.max())

# j'ajuste mon transformeur pour qu'il prenne la valeur du max sur le jeu de train
imputer_retard_livraison.fit(X_train[["retard_livraison"]])

# je remplace les valeurs manquantes du jeu de train par le max calculé (sur le jeu de train)
X_train[["retard_livraison"]] = imputer_retard_livraison.transform(X_train[["retard_livraison"]])

# je remplace les valeurs manquantes du jeu de test par le max calculé (sur le jeu de train)
X_test[["retard_livraison"]] = imputer_retard_livraison.transform(X_test[["retard_livraison"]])

In [12]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

In [13]:
model.fit(X_train,y_train)

LogisticRegression()

In [14]:
from sklearn.metrics import recall_score, accuracy_score, f1_score

recall = recall_score(y_train, model.predict(X_train))
acc = accuracy_score(y_train, model.predict(X_train))
f1 = f1_score(y_train, model.predict(X_train))

print(f"Pour le jeu d'entrainement: \n le recall est de {recall}, \n l'accuracy de {acc} \n le f1 score de {f1}")

Pour le jeu d'entrainement: 
 le recall est de 0.9833850487445258, 
 l'accuracy de 0.6400508420718144 
 le f1 score de 0.7593783721779915


In [15]:
recall = recall_score(y_test, model.predict(X_test))
acc = accuracy_score(y_test, model.predict(X_test))
f1 = f1_score(y_test, model.predict(X_test))

print(f"Pour le jeu de test: \n le recall est de {recall}, \n l'accuracy de {acc} \n le f1 score de {f1}")

Pour le jeu de test: 
 le recall est de 0.9834573304157549, 
 l'accuracy de 0.6441608622705781 
 le f1 score de 0.7625123002273422


### Plafonnement

In [16]:
from sklearn.model_selection import train_test_split
y = df['score']
X = df[["produit_recu","temps_livraison","retard_livraison"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [17]:
X_train["temps_livraison"] = X_train["temps_livraison"].apply(lambda x: 50 if x>50 else x)
X_test["temps_livraison"] = X_test["temps_livraison"].apply(lambda x: 50 if x>50 else x)

X_train["retard_livraison"] = X_train["retard_livraison"].apply(lambda x: 5 if x>5 else x)
X_test["retard_livraison"] = X_test["retard_livraison"].apply(lambda x: 5 if x>5 else x)
X_train["retard_livraison"] = X_train["retard_livraison"].apply(lambda x: -35 if x<-35 else x)
X_test["retard_livraison"] = X_test["retard_livraison"].apply(lambda x: -35 if x<-35 else x)



In [18]:
from sklearn.impute import SimpleImputer

# Je choisis la stratgéie de remplacer par le maximu
imputer_temps_livraison = SimpleImputer(strategy="constant",fill_value=X_train.temps_livraison.max())

# j'ajuste mon transformeur pour qu'il prenne la valeur du max sur le jeu de train
imputer_temps_livraison.fit(X_train[["temps_livraison"]])

# je remplace les valeurs manquantes du jeu de train par le max calculé (sur le jeu de train)
X_train[["temps_livraison"]] = imputer_temps_livraison.transform(X_train[["temps_livraison"]])

# je remplace les valeurs manquantes du jeu de test par le max calculé (sur le jeu de train)
X_test[["temps_livraison"]] = imputer_temps_livraison.transform(X_test[["temps_livraison"]])

In [19]:
# Je choisis la stratgéie de remplacer par le maximum
imputer_retard_livraison = SimpleImputer(strategy="constant",fill_value=X_train.retard_livraison.max())

# j'ajuste mon transformeur pour qu'il prenne la valeur du max sur le jeu de train
imputer_retard_livraison.fit(X_train[["retard_livraison"]])

# je remplace les valeurs manquantes du jeu de train par le max calculé (sur le jeu de train)
X_train[["retard_livraison"]] = imputer_retard_livraison.transform(X_train[["retard_livraison"]])

# je remplace les valeurs manquantes du jeu de test par le max calculé (sur le jeu de train)
X_test[["retard_livraison"]] = imputer_retard_livraison.transform(X_test[["retard_livraison"]])

In [20]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [21]:
from sklearn.metrics import recall_score, accuracy_score, f1_score

recall = recall_score(y_train, model.predict(X_train))
acc = accuracy_score(y_train, model.predict(X_train))
f1 = f1_score(y_train, model.predict(X_train))

print(f"Pour le jeu d'entrainement: \n le recall est de {recall}, \n l'accuracy de {acc} \n le f1 score de {f1}")

Pour le jeu d'entrainement: 
 le recall est de 0.9327259523337955, 
 l'accuracy de 0.6477915475055609 
 le f1 score de 0.7536407119614502


In [22]:
recall = recall_score(y_test, model.predict(X_test))
acc = accuracy_score(y_test, model.predict(X_test))
f1 = f1_score(y_test, model.predict(X_test))

print(f"Pour le jeu de test: \n le recall est de {recall}, \n l'accuracy de {acc} \n le f1 score de {f1}")

Pour le jeu de test: 
 le recall est de 0.9354048140043764, 
 l'accuracy de 0.6547867202196349 
 le f1 score de 0.7589120863513705


# EXERCICES

In [37]:
from sklearn.model_selection import train_test_split

y = df['score']
X = df[['order_status']]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [38]:
from sklearn.preprocessing import OneHotEncoder  

encoder = OneHotEncoder()

X_train_encoded = encoder.fit_transform(X_train[['order_status']])
X_test_encoded = encoder.transform(X_test[['order_status']])


In [42]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_encoded,y_train)

LogisticRegression()

In [43]:
from sklearn.metrics import recall_score, accuracy_score, f1_score

recall = recall_score(y_train, model.predict(X_train_encoded))
acc = accuracy_score(y_train, model.predict(X_train_encoded))
f1 = f1_score(y_train, model.predict(X_train_encoded))

print(f"Pour le jeu d'entrainement: \n le recall est de {recall}, \n l'accuracy de {acc} \n le f1 score de {f1}")

Pour le jeu d'entrainement: 
 le recall est de 0.995840760546643, 
 l'accuracy de 0.601131236097871 
 le f1 score de 0.7425359970463962


In [44]:

recall = recall_score(y_test, model.predict(X_test_encoded))
acc = accuracy_score(y_test, model.predict(X_test_encoded))
f1 = f1_score(y_test, model.predict(X_test_encoded))

print(f"Pour le jeu de test: \n le recall est de {recall}, \n l'accuracy de {acc} \n le f1 score de {f1}")


Pour le jeu de test: 
 le recall est de 0.9950109409190372, 
 l'accuracy de 0.6039961360516549 
 le f1 score de 0.7448321048321048


In [46]:
from sklearn.model_selection import train_test_split

y = df['score']
X = df[['order_status', 'produit_recu']]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [47]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(X_train_encoded,y_train)

LogisticRegression()

In [48]:
from sklearn.metrics import recall_score, accuracy_score, f1_score

recall = recall_score(y_train, model.predict(X_train_encoded))
acc = accuracy_score(y_train, model.predict(X_train_encoded))
f1 = f1_score(y_train, model.predict(X_train_encoded))

print(f"Pour le jeu d'entrainement: \n le recall est de {recall}, \n l'accuracy de {acc} \n le f1 score de {f1}")

Pour le jeu d'entrainement: 
 le recall est de 0.995840760546643, 
 l'accuracy de 0.601131236097871 
 le f1 score de 0.7425359970463962


In [49]:
from sklearn.metrics import recall_score, accuracy_score, f1_score

recall = recall_score(y_test, model.predict(X_test_encoded))
acc = accuracy_score(y_test, model.predict(X_test_encoded))
f1 = f1_score(y_test, model.predict(X_test_encoded))

print(f"Pour le jeu de test: \n le recall est de {recall}, \n l'accuracy de {acc} \n le f1 score de {f1}")

Pour le jeu de test: 
 le recall est de 0.9950109409190372, 
 l'accuracy de 0.6039961360516549 
 le f1 score de 0.7448321048321048


### Nettoyer category name

In [60]:
connection = sqlite3.connect("olist.db")

df_products = pd.read_sql_query("SELECT * FROM Products",connection)

df = pd.read_sql_query("SELECT * FROM TrainingDataset",connection)

df_orders = pd.read_sql_query('SELECT * FROM Orders', connection)

df_order_items = pd.read_sql_query("SELECT * FROM OrderItem",connection)

connection.close()

In [64]:
# Supposons que vous ayez les DataFrames suivants : TrainingDataset, Orders, OrderItems, Products

# 1. Fusionner TrainingDataset avec Orders pour récupérer order_id
df1 = pd.merge(df, df_orders , on='order_id', how='left')

# 2. Fusionner avec OrderItems pour récupérer product_id
df2 = pd.merge(df1, df_order_items, on = 'order_id', how= 'left')

# 3. Fusionner avec Products pour récupérer product_category_name

df3 = pd.merge(df2, df_products[['product_id', 'product_category_name']], on='product_id', how='left')



In [68]:
df3[['product_category_name']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112108 entries, 0 to 112107
Data columns (total 1 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   product_category_name  111383 non-null  object
dtypes: object(1)
memory usage: 876.0+ KB


## arbre de décision

In [ ]:
from sklearn.model_selection import train_test_split
y = df['score']
X = df[["produit_recu","temps_livraison","retard_livraison"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [ ]:
from sklearn.impute import SimpleImputer

# Je choisis la stratgéie de remplacer par le maximu
imputer_temps_livraison = SimpleImputer(strategy="constant",fill_value=X_train.temps_livraison.max())

# j'ajuste mon transformeur pour qu'il prenne la valeur du max sur le jeu de train
imputer_temps_livraison.fit(X_train[["temps_livraison"]])

# je remplace les valeurs manquantes du jeu de train par le max calculé (sur le jeu de train)
X_train[["temps_livraison"]] = imputer_temps_livraison.transform(X_train[["temps_livraison"]])

# je remplace les valeurs manquantes du jeu de test par le max calculé (sur le jeu de train)
X_test[["temps_livraison"]] = imputer_temps_livraison.transform(X_test[["temps_livraison"]])
# Je choisis la stratgéie de remplacer par le maximum
imputer_retard_livraison = SimpleImputer(strategy="constant",fill_value=X_train.retard_livraison.max())

# j'ajuste mon transformeur pour qu'il prenne la valeur du max sur le jeu de train
imputer_retard_livraison.fit(X_train[["retard_livraison"]])

# je remplace les valeurs manquantes du jeu de train par le max calculé (sur le jeu de train)
X_train[["retard_livraison"]] = imputer_retard_livraison.transform(X_train[["retard_livraison"]])

# je remplace les valeurs manquantes du jeu de test par le max calculé (sur le jeu de train)
X_test[["retard_livraison"]] = imputer_retard_livraison.transform(X_test[["retard_livraison"]])
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=2)
model.fit(X_train,y_train)
DecisionTreeClassifier(max_depth=2)

DecisionTreeClassifier(max_depth=2)

In [ ]:
from sklearn.metrics import recall_score, accuracy_score, f1_score

recall = recall_score(y_train, model.predict(X_train))
acc = accuracy_score(y_train, model.predict(X_train))
f1 = f1_score(y_train, model.predict(X_train))

print(f"Pour le jeu d'entrainement: \n le recall est de {recall}, \n l'accuracy de {acc} \n le f1 score de {f1}")

Pour le jeu d'entrainement: 
 le recall est de 0.9829669241433947, 
 l'accuracy de 0.6427073403241182 
 le f1 score de 0.7606518851537755


In [ ]:
recall = recall_score(y_test, model.predict(X_test))
acc = accuracy_score(y_test, model.predict(X_test))
f1 = f1_score(y_test, model.predict(X_test))

print(f"Pour le jeu de test: \n le recall est de {recall}, \n l'accuracy de {acc} \n le f1 score de {f1}")

Pour le jeu de test: 
 le recall est de 0.9828446389496718, 
 l'accuracy de 0.6459911535919467 
 le f1 score de 0.7633323136535128
